In [264]:
# ==========================================
# Data Loading & Cleaning
# ==========================================
import pandas as pd
import numpy as np
import glob
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import NearestNeighbors

# 1. Load & gabungkan CSV
files = glob.glob("data/raw/*.csv")
df = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)
df.columns = (
    df.columns.str.strip().str.lower()
              .str.replace(" ", "_")
              .str.replace(r"\(.*?\)", "", regex=True)
              .str.replace("/", "_")
              .str.replace("__", "_")
              .str.replace("-", "_")
              .str.strip("_")
)
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.dropna(subset=['date','country','area'])
df.describe()

,date,ammonia,biochemical_oxygen_demand,dissolved_oxygen,orthophosphate,ph,temperature,nitrogen,nitrate,ccme_values
count,25651,25651.000000,25651.000000,25651.000000,25651.000000,25651.000000,25651.000000,25651.000000,25651.000000,25651.000000
mean,2009-03-27 06:11:48.182916864,0.180378,1.180863,8.490642,0.063232,7.951231,21.726652,0.123046,1.281441,95.993955
min,1968-12-01 00:00:00,0.000000,0.000000,0.058333,0.000000,2.100000,0.000000,0.000000,0.000000,37.659232
25%,2004-11-10 00:00:00,0.027000,0.500000,6.100000,0.007000,7.800000,18.000000,0.014000,0.042000,93.180250
50%,2009-09-01 00:00:00,0.056000,0.800000,8.200000,0.014000,8.000000,22.200000,0.020000,0.094000,100.000000
75%,2013-12-12 00:00:00,0.130000,1.366685,11.684103,0.029000,8.100000,26.400000,0.052000,0.240000,100.000000
max,2021-12-02 00:00:00,63.836364,231.108000,16.930476,35.337500,19.445000,89.000000,41.784200,150.000000,100.000000
std,NaN,1.158218,2.867754,2.813504,0.777032,0.351018,5.485493,0.593947,5.255924,6.256453


In [265]:
# 2. Data Cleaning (domain-based filtering)
df = df[(df['ph'] >= 4) & (df['ph'] <= 10)]
df = df[(df['temperature'] >= 0) & (df['temperature'] <= 40)]
df = df[df['ammonia'] <= 10]
df = df[df['biochemical_oxygen_demand'] <= 50]
df = df[df['orthophosphate'] <= 1]
df = df[df['nitrogen'] <= 10]
df = df[df['nitrate'] <= 50]

num_cols = df.select_dtypes(include='number').columns.tolist()
num_cols = [c for c in num_cols if c not in ['ccme_values', 'ccme_wqi_code']]
df.describe()

,date,ammonia,biochemical_oxygen_demand,dissolved_oxygen,orthophosphate,ph,temperature,nitrogen,nitrate,ccme_values
count,25452,25452.000000,25452.000000,25452.000000,25452.000000,25452.000000,25452.000000,25452.000000,25452.000000,25452.000000
mean,2009-03-24 23:03:21.980198144,0.122760,1.113556,8.482102,0.028767,7.952216,21.785461,0.111060,1.091613,96.267933
min,1968-12-01 00:00:00,0.000000,0.000000,0.058333,0.000000,4.100000,0.000000,0.000000,0.000000,51.076024
25%,2004-11-10 00:00:00,0.027000,0.500000,6.100000,0.007000,7.800000,18.000000,0.014000,0.041000,93.181497
50%,2009-08-07 00:00:00,0.055000,0.800000,8.200000,0.014000,8.000000,22.300000,0.020000,0.093000,100.000000
75%,2013-12-09 00:00:00,0.130000,1.300000,11.684103,0.028000,8.100000,26.400000,0.051000,0.230000,100.000000
max,2021-12-02 00:00:00,9.903110,42.100000,16.100000,0.960000,9.890000,32.100000,9.050000,50.000000,100.000000
std,NaN,0.376039,1.219039,2.818957,0.059311,0.329495,5.435426,0.451301,3.973857,5.237777


In [266]:
# 3. Outlier removal (IQR)
Q1 = df[num_cols].quantile(0.25)
Q3 = df[num_cols].quantile(0.75)
IQR = Q3 - Q1
df_clean = df[~((df[num_cols] < (Q1 - 1.5*IQR)) | (df[num_cols] > (Q3 + 1.5*IQR))).any(axis=1)]
df_clean.describe()

,date,ammonia,biochemical_oxygen_demand,dissolved_oxygen,orthophosphate,ph,temperature,nitrogen,nitrate,ccme_values
count,19662,19662.000000,19662.000000,19662.000000,19662.000000,19662.000000,19662.000000,19662.000000,19662.000000,19662.000000
mean,2009-05-26 01:43:29.093683456,0.074476,0.788024,8.296040,0.015104,8.005592,22.791708,0.023774,0.099032,97.453196
min,1976-10-04 00:00:00,0.002576,0.100000,0.100000,0.000000,7.400000,6.172000,0.000000,0.000000,86.329554
25%,2005-01-12 00:00:00,0.024000,0.500000,6.000000,0.007000,7.900000,19.400000,0.012000,0.029000,93.190379
50%,2009-05-22 00:00:00,0.047000,0.700000,7.500000,0.012000,8.000000,23.000000,0.019000,0.077000,100.000000
75%,2013-11-01 00:00:00,0.110000,1.000000,11.684103,0.021000,8.200000,26.400000,0.030000,0.130000,100.000000
max,2017-12-10 00:00:00,0.280000,2.500000,12.700000,0.059000,8.533330,31.800000,0.100000,0.510000,100.000000
std,NaN,0.065670,0.468061,2.875592,0.010520,0.210308,4.050359,0.018606,0.093636,3.311349


In [267]:
# 4. CCME WQI calculation
OBJECTIVES = {
    'ammonia': 0.5, 'biochemical_oxygen_demand': 5.0, 'dissolved_oxygen': 6.5,
    'orthophosphate': 0.1, 'ph': None, 'temperature': 35.0, 'nitrogen': 1.0, 'nitrate': 10.0
}
HIGHER_IS_WORSE = {
    'ammonia': True, 'biochemical_oxygen_demand': True, 'dissolved_oxygen': False,
    'orthophosphate': True, 'ph': None, 'temperature': True, 'nitrogen': True, 'nitrate': True
}
def ccme_wqi_from_row(row, params):
    def excursion(v,obj,higher=True):
        if pd.isna(v): return 0
        return (v/obj-1) if (higher and v>obj) else ((obj/v-1) if (not higher and v<obj) else 0)
    excursions = []
    failed_vars = set()
    failed_tests = 0
    total_tests = len(params)
    for p in params:
        value = row[p]
        if p=="ph":
            low, high = 6.5, 8.5
            exc = (low/value-1) if value<low else ((value/high-1) if value>high else 0)
        else:
            obj = OBJECTIVES.get(p)
            exc = excursion(value,obj,HIGHER_IS_WORSE[p]) if obj else 0
        if exc>0:
            excursions.append(exc)
            failed_vars.add(p)
            failed_tests +=1
    F1 = len(failed_vars)/total_tests*100
    F2 = failed_tests/total_tests*100
    normalized_exc = np.sum(excursions)/total_tests
    F3 = normalized_exc/(0.01*normalized_exc+0.01) if normalized_exc>0 else 0
    score = 100 - (np.sqrt(F1**2+F2**2+F3**2)/1.732)
    score = np.clip(score,0,100)
    if score>=95: cat='Excellent'
    elif score>=80: cat='Good'
    elif score>=65: cat='Fair'
    elif score>=45: cat='Marginal'
    else: cat='Poor'
    return score, cat

df_clean[['ccme_values','ccme_wqi']] = df_clean.apply(
    lambda r: pd.Series(ccme_wqi_from_row(
        r, ["ammonia","biochemical_oxygen_demand","dissolved_oxygen",
            "orthophosphate","ph","temperature","nitrogen","nitrate"]
    )), axis=1
)
df_clean.describe()

C:\Users\Rafael Po\AppData\Local\Temp\ipykernel_7640\4181548535.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[['ccme_values','ccme_wqi']] = df_clean.apply(


,date,ammonia,biochemical_oxygen_demand,dissolved_oxygen,orthophosphate,ph,temperature,nitrogen,nitrate,ccme_values
count,19662,19662.000000,19662.000000,19662.000000,19662.000000,19662.000000,19662.000000,19662.000000,19662.000000,19662.000000
mean,2009-05-26 01:43:29.093683456,0.074476,0.788024,8.296040,0.015104,8.005592,22.791708,0.023774,0.099032,96.439389
min,1976-10-04 00:00:00,0.002576,0.100000,0.100000,0.000000,7.400000,6.172000,0.000000,0.000000,47.673409
25%,2005-01-12 00:00:00,0.024000,0.500000,6.000000,0.007000,7.900000,19.400000,0.012000,0.029000,89.776152
50%,2009-05-22 00:00:00,0.047000,0.700000,7.500000,0.012000,8.000000,23.000000,0.019000,0.077000,100.000000
75%,2013-11-01 00:00:00,0.110000,1.000000,11.684103,0.021000,8.200000,26.400000,0.030000,0.130000,100.000000
max,2017-12-10 00:00:00,0.280000,2.500000,12.700000,0.059000,8.533330,31.800000,0.100000,0.510000,100.000000
std,NaN,0.065670,0.468061,2.875592,0.010520,0.210308,4.050359,0.018606,0.093636,5.125450


In [268]:
# ==========================================
# 5. Synthetic Data Generation (Poor)
# ==========================================
def generate_synthetic_poor(n, df_reference):
    rows=[]
    for _ in range(n):
        r = {c: np.random.uniform(df_reference[c].min(), df_reference[c].max()) for c in num_cols}
        r.update({
            "country":"Synthetic", "area":"Synthetic",
            "date": df_reference['date'].sample(1).iloc[0]  # random dari asli
        })
        rows.append(r)
    df_synth = pd.DataFrame(rows)
    df_synth[['ccme_values','ccme_wqi']] = df_synth.apply(
        lambda r: pd.Series(ccme_wqi_from_row(
            r, ["ammonia","biochemical_oxygen_demand","dissolved_oxygen",
                "orthophosphate","ph","temperature","nitrogen","nitrate"]
        )), axis=1
    )
    return df_synth

# Hitung jumlah synthetic Poor yang dibutuhkan
target_size = df_clean['ccme_wqi'].value_counts().max()
current_poor = df_clean[df_clean['ccme_wqi']=='Poor'].shape[0]
need_poor = target_size - current_poor
df_synth_poor = generate_synthetic_poor(need_poor, df_clean) if need_poor>0 else pd.DataFrame()

df_balanced = pd.concat([df_clean, df_synth_poor], ignore_index=True)
df_balanced.describe()

,date,ammonia,biochemical_oxygen_demand,dissolved_oxygen,orthophosphate,ph,temperature,nitrogen,nitrate,ccme_values
count,32783,32783.000000,32783.000000,32783.000000,32783.000000,32783.000000,32783.000000,32783.000000,32783.000000,32783.000000
mean,2009-05-29 09:13:03.735472640,0.100802,0.991302,7.549576,0.020875,7.988851,21.222834,0.034264,0.161754,94.436435
min,1976-10-04 00:00:00,0.002576,0.100000,0.100000,0.000000,7.400000,6.172000,0.000000,0.000000,46.393224
25%,2005-02-03 00:00:00,0.032000,0.500000,5.209708,0.008000,7.800000,17.400000,0.014440,0.050000,89.644621
50%,2009-06-01 00:00:00,0.078000,0.800000,7.200000,0.016641,8.000000,21.800000,0.024000,0.110000,100.000000
75%,2013-11-03 00:00:00,0.160000,1.400000,11.684103,0.031000,8.200000,26.100000,0.051677,0.250000,100.000000
max,2017-12-10 00:00:00,0.280000,2.500000,12.700000,0.059000,8.533330,31.800000,0.100000,0.510000,100.000000
std,NaN,0.078796,0.621313,3.338553,0.015280,0.263313,5.950686,0.026510,0.141035,8.175832


In [269]:
# ==========================================
# 6. Synthetic Good (SMOTE-like)
# ==========================================
df_good = df_balanced[df_balanced['ccme_wqi']=='Good']
need_good = target_size - len(df_good)

X_good = df_good[num_cols].values
k = min(5, len(X_good)-1)
nn = NearestNeighbors(n_neighbors=k).fit(X_good)

synthetic_good=[]
for _ in range(need_good):
    idx = np.random.randint(0,len(X_good))
    distances, neighbors = nn.kneighbors([X_good[idx]])
    neighbor_idx = np.random.choice(neighbors[0][1:])
    alpha = np.random.rand()
    new_sample = X_good[idx] + alpha*(X_good[neighbor_idx]-X_good[idx])
    synthetic_good.append(new_sample)

synthetic_good = pd.DataFrame(synthetic_good, columns=num_cols)
synthetic_good['ccme_wqi']='Good'
synthetic_good['country']='Synthetic'
synthetic_good['area']='Synthetic'
synthetic_good['date'] = df_clean['date'].sample(len(synthetic_good)).values

df_balanced = pd.concat([df_balanced, synthetic_good], ignore_index=True)
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

In [272]:
# ==========================================
# 7. Label Encoding & Scaling
# ==========================================
le = LabelEncoder()
df_balanced['label'] = le.fit_transform(df_balanced['ccme_wqi'])

scaled_cols = ['ammonia','biochemical_oxygen_demand','dissolved_oxygen',
               'orthophosphate','ph','temperature','nitrogen','nitrate']

scaler = StandardScaler()
df_balanced.to_csv("data/Cleaned_dataset.csv", index=False)
df_balanced[scaled_cols] = scaler.fit_transform(df_balanced[scaled_cols])
df_balanced.describe()


,date,ammonia,biochemical_oxygen_demand,dissolved_oxygen,orthophosphate,ph,temperature,nitrogen,nitrate,ccme_values,label
count,34200,3.420000e+04,3.420000e+04,3.420000e+04,3.420000e+04,3.420000e+04,3.420000e+04,3.420000e+04,3.420000e+04,32783.000000,34200.000000
mean,2009-06-01 07:07:52.421052416,-1.153074e-17,-3.635818e-18,2.929431e-17,9.972530e-18,-2.223043e-17,1.807521e-17,-3.012535e-17,3.599460e-17,94.436435,0.844094
min,1976-10-04 00:00:00,-1.255331e+00,-1.436591e+00,-2.203017e+00,-1.377711e+00,-2.242452e+00,-2.532332e+00,-1.302420e+00,-1.151740e+00,46.393224,0.000000
25%,2005-02-06 00:00:00,-8.787525e-01,-7.917388e-01,-7.010967e-01,-8.516678e-01,-7.189095e-01,-6.482259e-01,-7.339795e-01,-7.888529e-01,89.644621,0.000000
50%,2009-06-01 00:00:00,-2.824197e-01,-3.080995e-01,-1.303671e-01,-2.598693e-01,4.286151e-02,1.068931e-01,-3.929151e-01,-3.690428e-01,100.000000,0.000000
75%,2013-11-03 00:00:00,7.487456e-01,6.591792e-01,1.231057e+00,6.607061e-01,8.046325e-01,8.222137e-01,6.623757e-01,6.271169e-01,100.000000,2.000000
max,2017-12-10 00:00:00,2.276398e+00,2.432523e+00,1.581822e+00,2.501857e+00,2.074238e+00,1.768155e+00,2.487185e+00,2.477128e+00,100.000000,3.000000
std,NaN,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,8.175832,0.992423


In [273]:
# ==========================================
# 8. Cek distribusi akhir
# ==========================================
print(df_balanced['ccme_wqi'].value_counts())
print(df_balanced.describe())
print(le.classes_)
df_balanced.to_csv("data/Preprocessed_dataset.csv", index=False)

ccme_wqi
Excellent    19433
Good         13121
Fair          1156
Marginal       490
Name: count, dtype: int64
                                date       ammonia  biochemical_oxygen_demand  \
count                          34200  3.420000e+04               3.420000e+04   
mean   2009-06-01 07:07:52.421052416 -1.153074e-17              -3.635818e-18   
min              1976-10-04 00:00:00 -1.255331e+00              -1.436591e+00   
25%              2005-02-06 00:00:00 -8.787525e-01              -7.917388e-01   
50%              2009-06-01 00:00:00 -2.824197e-01              -3.080995e-01   
75%              2013-11-03 00:00:00  7.487456e-01               6.591792e-01   
max              2017-12-10 00:00:00  2.276398e+00               2.432523e+00   
std                              NaN  1.000015e+00               1.000015e+00   

       dissolved_oxygen  orthophosphate            ph   temperature  \
count      3.420000e+04    3.420000e+04  3.420000e+04  3.420000e+04   
mean       2.9294

In [ ]:
import joblib

le = LabelEncoder()
df_balanced['label'] = le.fit_transform(df_balanced['ccme_wqi'])

# simpan encoder
joblib.dump(le, 'notebooks/label_encoder.pkl')


['label_encoder.pkl']